In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

class Transform:
  def __init__(self):
    pass

  def transform(self,inputDF):
    pass


class AirpodsAfterIphoneTransformer(Transform):

  def transform(self, inputDF):

    "customers who have bought Airpods after buying the iPhone"

    transactionInputDF= inputDF.get("transactionInputDF")

    print("transactionInputDF in transform")

    transactionInputDF.show()

    windowspec= Window.partitionBy('customer_id').orderBy('transaction_date')

    transformedDF= transactionInputDF.withColumn('next_product_name', lead('product_name').over(windowspec))

    print('Airpods after buying iPhones')
    transformedDF.orderBy('customer_id','transaction_date','product_name').show()

    filteredDF= transformedDF.filter( (col('product_name')=='iPhone')& (col('next_product_name')=='AirPods') )

    print('filtered_products')

    filteredDF.orderBy('customer_id','transaction_date','product_name').show()

    customerInputDF= inputDF.get('customerInputDF')

    customerInputDF.show()

    joinedDF= customerInputDF.join(broadcast(filteredDF),customerInputDF['customer_id']==filteredDF['customer_id'])

    print('joined DF')
    return joinedDF.select(customerInputDF['customer_id'],'customer_name','location')
  

class OnlyAirpodsAndIphone(Transform):
  def transform(self,inputDF):
    """
    Customer who have bought only iPhone and Airpods 

    """
    transactionInputDF= inputDF.get("transactionInputDF")

    print("transactionInputDF in transform")

    groupedDF= transactionInputDF.groupBy('customer_id').agg(collect_set('product_name').alias('products'))

    print('Grouped Dataframe')

    groupedDF.show()

    filteredDF= groupedDF.filter( (array_contains(col('products'),'iPhone'))& (array_contains(col('products'),'AirPods'))
                                     &(size(col('products'))==2) )

    print('only Airpods and iPhones')

    filteredDF.show()

    customerInputDF= inputDF.get('customerInputDF')

    customerInputDF.show()

    joinedDF= customerInputDF.join(broadcast(filteredDF),customerInputDF['customer_id']==filteredDF['customer_id'])

    joinedDF.show()
    return joinedDF.select(customerInputDF['customer_id'],'customer_name','location')
    


    

    
    


